Required dependecies

In [ ]:
!pip install --upgrade pip
!pip install --upgrade tensorflow
!pip install -q kaggle
!pip install zipfile36

In [ ]:
!mkdir checkpoints
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d moltean/fruits

mkdir: cannot create directory ‘checkpoints’: File exists
 99% 1.06G/1.08G [00:08<00:00, 112MB/s] 
100% 1.08G/1.08G [00:08<00:00, 139MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile('fruits.zip', 'r') as zipObj:
   zipObj.extractall()

# **Extraction of types from "fruits-360" dataset**

In [ ]:
!mkdir fruits-360-type
!mkdir fruits-360-type/Test
!mkdir fruits-360-type/Training

import shutil
import os

soruce_dir_test="fruits-360_dataset/fruits-360/Test/"
soruce_dir_training="fruits-360_dataset/fruits-360/Training/"
test_dir="fruits-360-type/Test/"
train_dir="fruits-360-type/Training/"

def extract_types(directory, destination):
  type_counter={'apple':0,'banana':0,'cherry':0,'grape':0,'peach':0,'pear':0,'pepper':0,'plum':0,'potato':0,'tomato':0}
  for label_type in ['apple','banana','cherry','grape','peach','pear','pepper','plum','potato','tomato']:
    for label_variety in os.listdir(directory):
      if(label_variety.lower().find(label_type)==0 and label_variety.lower().find("grapefruit")!=0):
          image_list=os.listdir(os.path.join(directory,label_variety))
          for image in image_list:
              shutil.copy(os.path.join(directory,label_variety,image), os.path.join(destination,label_type,f"{label_type}_{type_counter[label_type]}.jpg"))
              type_counter[label_type]+=1
      else:
        continue

for label in ['apple','banana','cherry','grape','peach','pear','pepper','plum','potato','tomato']:
  os.system(f"mkdir {os.path.join(test_dir,label)}")
  os.system(f"mkdir {os.path.join(train_dir,label)}")

extract_types(soruce_dir_test, test_dir)
extract_types(soruce_dir_training, train_dir)

# **Classification based on fruit and vegetable types**

In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow import keras
import tensorflow.keras.preprocessing.image
from tensorflow.keras import activations
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

train_data=[]
train_label=[]
test_data=[]
test_label=[]
epochs=10
batch_size=16
optimizer="adam"
img_size = (32, 32)
shape = (32, 32, 3)
test_dir = 'fruits-360-type/Test'
train_dir = 'fruits-360-type/Training'
metrics=['accuracy']

labels = os.listdir(train_dir)
num_classes = len(labels)

def calculate_class_weights(train_label):
    train_size=len(train_label)
    weights={}
    for i in range(10):
        weights[i]=0
    for i in train_label:
        weights[i]=weights[i]+1
    for i in range(num_classes):
        weights[i]=train_size/(weights[i]*2.)
    return weights

def load_data(dir):
    i=0
    data=[]
    data_label=[]
    for label in labels:
        images=os.listdir(os.path.join(dir,label))
        for image in images:
            image_path=os.path.join(dir,label,image)
            image=load_img(image_path,color_mode='rgb', target_size=img_size)
            array_image=img_to_array(image)
            array_image=array_image/255.0
            data.append(array_image)
            data_label.append(i)
        i=i+1
    tmp=list(zip(data, data_label))
    tmp=random(tmp)
    data, data_label=zip(*tmp)
    return data, data_label

print("Loading training images...")
train_data, train_label=load_data(train_dir)
train_data=np.array(train_data)
train_label=np.array(train_label)
print("Loading test images...")
test_data, test_label=load_data(test_dir)
test_data=np.array(test_data)
test_label=np.array(test_label)
checkpoint_dir = "checkpoints/32_64_128_Filters_512_Dense.h5"
callbacks=[ModelCheckpoint(checkpoint_dir, monitor='loss', verbose=0, save_best_only=True,save_weights_only=False, mode='auto', save_freq='epoch')]
network=tf.keras.models.Sequential()
network.add(Input(shape=shape))
network.add(Conv2D(32, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(64, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(128, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Flatten())
network.add(Dense(512, activation="softmax"))
network.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=metrics)

print(network.summary())
weights=calculate_class_weights(train_label)
history = network.fit(x=train_data,y=train_label,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      callbacks=callbacks)
train_loss, train_accuracy = network.evaluate(x=train_data,y=train_label,steps=(len(train_data)//batch_size)+1, verbose=1)
test_loss, test_accuracy = network.evaluate(x=test_data, y=test_label,steps=(len(test_data)//batch_size)+1, verbose=1)
print(f"Train accuracy = {train_accuracy}")
print(f"Test accuracy = {test_accuracy}")

Loading training images...
Loading test images...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flat

# **Classification based on fruit and vegetable varieties**

In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import  ModelCheckpoint
import coremltools as ct

train_data=[]
train_label=[]
test_data=[]
test_label=[]
epochs=20
batch_size=16
optimizer="adam"
img_size = (32, 32)
shape = (32, 32, 3)
test_dir = 'fruits-360_dataset/fruits-360/Test'
train_dir = 'fruits-360_dataset/fruits-360/Training'
metrics=['accuracy']

labels = os.listdir(train_dir)
print(labels)
num_classes = len(labels)

def calculate_class_weights(train_label, num_classes):
    train_size=len(train_label)
    weights={}
    for i in range(num_classes):
        weights[i]=0
    for i in train_label:
        weights[i]=weights[i]+1
    for i in range(num_classes):
        weights[i]=train_size/(weights[i]*2.)
    return weights

def load_data(dir, labels):
    i=0
    data=[]
    data_label=[]
    for label in labels:
        images=os.listdir(os.path.join(dir,label))
        for image in images:
            image_path=os.path.join(dir,label,image)
            image=load_img(image_path,color_mode='rgb', target_size=img_size)
            array_image=img_to_array(image)
            array_image=array_image/255.0
            data.append(array_image)
            data_label.append(i)
        i=i+1
    tmp=list(zip(data, data_label))
    data, data_label=zip(*tmp)
    return data, data_label

print("Loading training images...")
train_data, train_label=load_data(train_dir, labels)
train_data=np.array(train_data)
train_label=np.array(train_label)
print("Loading test images...")
test_data, test_label=load_data(test_dir, labels)
test_data=np.array(test_data)
test_label=np.array(test_label)
checkpoint_dir = "checkpoints/32_64_Filters_512_Dense_50%_Dropout.ckpt"
callbacks=[ModelCheckpoint(checkpoint_dir, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True)]

network=tf.keras.models.Sequential()
network.add(Input(shape=shape))
network.add(Conv2D(32, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(64, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Flatten())
network.add(Dense(512, activation="relu"))
network.add(Dropout(0.5))
network.add(Dense(num_classes, activation="softmax"))
network.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=metrics)
print(network.summary())
weights=calculate_class_weights(train_label, num_classes)
history = network.fit(x=train_data,y=train_label,
                        batch_size=batch_size,
                        epochs=epochs,
                        class_weight=weights,
                        verbose=1,
                        callbacks=callbacks)
network.load_weights(checkpoint_dir)
train_loss, train_accuracy = network.evaluate(x=train_data,y=train_label, verbose=1)
test_loss, test_accuracy = network.evaluate(x=test_data, y=test_label, verbose=1)
predicted=network.predict(test_data, batch_size=16, verbose=1)
predicted = predicted.argmax(axis=-1)
print(f"Train accuracy = {train_accuracy}")
print(f"Test accuracy = {test_accuracy}")